## Milestone Action Items

Download the data set about Value of Energy Cost Saving Program for businesses in New York City (under the "Export" option, there is a way to retrieve a CSV file). Answer the following questions.

1. How many different companies are represented in the data set?
2. What is the total number of jobs created for businesses in Queens?
3. How many different unique email domains names are there in the data set?
4. Considering only NTAs with at least 5 listed businesses, what is the average total savings and the total jobs created for each NTA?
5. Save your result for the previous question as a CSV file.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Value_of_Energy_Cost_Savings_Program_Savings_for_Businesses_-_FY2020.csv')

In [5]:
df.dtypes

Company Name                                  object
company contact                               object
company email                                 object
company phone                                 object
Address                                       object
City                                          object
State                                         object
Postcode                                       int64
Industry                                      object
Industry descr                                object
Company Type                                  object
Current fulltime                             float64
Job created                                  float64
Job retain                                   float64
Effective Date                                object
Total Savings                                float64
Savings from beginning receiving benefits    float64
Borough                                       object
Latitude                                     f

In [6]:
df.head()

,Company Name,company contact,company email,company phone,Address,City,State,Postcode,Industry,Industry descr,...,Savings from beginning receiving benefits,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1 State Street,David Neiburg,eskoblo@energyspec.com,7186779077,1114 Avenue J,Brooklyn,NY,11230,Commercial,35 story building,...,349.78,Brooklyn,40.624784,-73.964920,314.0,44.0,534.0,3179818.0,3.067130e+09,Midwood
1,100 Church LLC,Edward A. Carroll,NaN,2125663000,100 Church Street,NY,NY,10007,Commercial,21 Floors,...,4423698.05,New York,40.712968,-74.009490,101.0,1.0,21.0,1001409.0,1.001250e+09,SoHo-TriBeCa-Civic Center-Little Italy
2,100 Wall Investments,Jerry Branam,NaN,2122487160,96 Wall Street,New York,NY,10005,Landlord,NaN,...,1814542.62,New York,40.705052,-74.007199,101.0,1.0,7.0,1000875.0,1.000380e+09,Battery Park City-Lower Manhattan
3,1-10 Bush Terminal Owner LP,Andrew Kimball,akimball@industrycity.com,3474178429,"220 36th St, Suite 2A",Brooklyn,NY,11232,Landlord,Special Eligible Premises for eligible businesses,...,6755.31,Brooklyn,40.656061,-74.007334,307.0,38.0,2.0,3398687.0,3.006950e+09,Sunset Park West
4,"120 Broadway Acquisitions JV, LLC (Ldmk)",Carl Letich,clettich@silvprop.com,2124064020,120 Broadway,New York,NY,10271,Commercial,NaN,...,3293055.08,New York,40.708546,-74.011041,101.0,1.0,7.0,1001026.0,1.000478e+09,Battery Park City-Lower Manhattan


### How many different companies are represented in the data set?

In [7]:
df['Company Name'].nunique()

439

### What is the total number of jobs created for businesses in Queens?

In [26]:
queens_created = df[(df['Job created']>0) & (df['Borough']=='Queens')]['Job created']
queens_created

29     10.0
37     60.0
137    10.0
324     2.0
433    20.0
Name: Job created, dtype: float64

In [27]:
sum(queens_created)

102.0

### How many different unique email domains names are there in the data set?

In [78]:
def get_domain(address):
    """
    Take in an e-mail address and return the domain
    Need to account for improperly entered addresses (e.g. missing domain) 
    """
    raw_addr = address.split('@')
    if len(raw_addr)>1:
        return(raw_addr[1])

# Remove missing emails
emails = df['company email'].dropna()

In [79]:
domains = emails.apply(get_domain)

In [81]:
len(domains.unique())

357

### Considering only NTAs with at least 5 listed businesses, what is the average total savings and the total jobs created for each NTA?

In [130]:
five_plus = df.groupby('NTA').count()['Company Name']>=5

fiveplus_ntas = five_plus.index[five_plus==True].values

df_5plus = df[df['NTA'].isin(fiveplus_ntas)]

In [173]:
mean_ts = df_5plus.groupby('NTA').mean()['Total Savings']
mean_ts

NTA
Battery Park City-Lower Manhattan                   469890.796667
Bushwick South                                       26140.455000
Carroll Gardens-Columbia Street-Red Hook             24337.390000
College Point                                        17159.372857
Crown Heights North                                   4036.298889
DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill     59950.340000
East New York                                        48433.093333
East Williamsburg                                    13990.626364
Greenpoint                                            9753.649286
Hunters Point-Sunnyside-West Maspeth                 23931.398933
Hunts Point                                          39246.016154
Manhattanville                                        2760.888889
Maspeth                                               6658.283333
Mott Haven-Port Morris                               17843.937857
Queensbridge-Ravenswood-Long Island City             61830.770625
SoHo-T

In [175]:
total_jobs = df_5plus.groupby('NTA').sum()['Job created']
total_jobs

NTA
Battery Park City-Lower Manhattan                    0.0
Bushwick South                                      10.0
Carroll Gardens-Columbia Street-Red Hook             0.0
College Point                                        2.0
Crown Heights North                                  0.0
DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill     0.0
East New York                                       15.0
East Williamsburg                                    0.0
Greenpoint                                           0.0
Hunters Point-Sunnyside-West Maspeth                 0.0
Hunts Point                                         12.0
Manhattanville                                       0.0
Maspeth                                             10.0
Mott Haven-Port Morris                              10.0
Queensbridge-Ravenswood-Long Island City             0.0
SoHo-TriBeCa-Civic Center-Little Italy               0.0
Springfield Gardens South-Brookville                 0.0
Steinway                   

In [181]:
df_output = pd.DataFrame(list(zip(mean_ts, total_jobs)), columns=['Mean Total Savings','Total Jobs Created'], index=total_jobs.index)

In [182]:
df_output

,Mean Total Savings,Total Jobs Created
NTA,,
Battery Park City-Lower Manhattan,469890.796667,0.0
Bushwick South,26140.455000,10.0
Carroll Gardens-Columbia Street-Red Hook,24337.390000,0.0
College Point,17159.372857,2.0
Crown Heights North,4036.298889,0.0
DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,59950.340000,0.0
East New York,48433.093333,15.0
East Williamsburg,13990.626364,0.0
Greenpoint,9753.649286,0.0


In [183]:
df_output.to_csv('output.csv')